In [4]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform, td_approximants
from pycbc.detector import Detector
import matplotlib.pyplot as plt
import numpy as np
import gwpy
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv
import pycbc.noise
import pycbc.psd
from pycbc.filter import matched_filter

In [5]:
no_of_params = 2
directory = "C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-val-data-files/Parameter Estimation/"+str(no_of_params)+"_parameters/"

In [6]:
bbh_two_mass_distributions = distributions.Uniform(mass1=(10, 50),mass2=(10, 50))
bbh_two_mass_samples = bbh_two_mass_distributions.rvs(size=1000)

In [7]:
start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
bbh_data_targets = np.zeros((len(bbh_two_mass_samples), no_of_params))
os.makedirs(directory, exist_ok=True)
os.makedirs(directory+"merged_bbh_noise_signal/", exist_ok=True)

for i in tqdm(range(len(bbh_two_mass_samples))):

    m1 = max(bbh_two_mass_samples[i][0], bbh_two_mass_samples[i][1])
    m2 = min(bbh_two_mass_samples[i][0], bbh_two_mass_samples[i][1])

    bbh_data_targets[i][0] = m1
    bbh_data_targets[i][1] = m2

    hp1, hc1 = get_td_waveform(approximant="SEOBNRv2",                                
                         mass1=m1,
                         mass2=m2,
                         delta_t=1.0/4096,
                         f_lower=40)

    
    bbh_signal = TimeSeries.from_pycbc(hp1)
    st1 = np.random.randint(0, 8)
    bbh_signal.t0 = start_times_bbh[st1]
    bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)

    data = noise.inject(bbh_signal)
    data *= 1e-17

    data.write(directory + "merged_bbh_noise_signal/merged_noise_signal_"+str(i)+".txt")
np.savetxt(directory + "val_Final_BBH_Merged_Noise_Signal_Targets_"+str(no_of_params)+"_parameters.csv", bbh_data_targets, delimiter = ",")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
path = directory + "merged_bbh_noise_signal/"
files= os.listdir(path)
f = open(directory + 'val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS_'+str(no_of_params)+'_parameters.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
bns_two_mass_distributions = distributions.Uniform(mass1=(1, 2),mass2=(1, 2))
bns_two_mass_samples = bns_two_mass_distributions.rvs(size=1000)

In [10]:
start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]
bns_data_targets = np.zeros((len(bns_two_mass_samples), no_of_params))
os.makedirs(directory+"merged_bns_noise_signal/", exist_ok=True)


for i in tqdm(range(len(bns_two_mass_samples))):

    m1 = max(bns_two_mass_samples[i][0], bns_two_mass_samples[i][1])
    m2 = min(bns_two_mass_samples[i][0], bns_two_mass_samples[i][1])
    
    bns_data_targets[i][0] = m1
    bns_data_targets[i][1] = m2

    hp2, hc2 = get_td_waveform(approximant="IMRPhenomPv2_NRTidal", 
                         mass1=m1,
                         mass2=m2,
                         delta_t=1.0/4096,
                         f_lower=40)


    # Extract the last 1 sec from the BNS signal
    t = hp2.get_end_time()
    hp2 = hp2.time_slice(t-1, t)

    bns_signal = TimeSeries.from_pycbc(hp2)
    st2 = np.random.randint(0, 7)
    bns_signal.t0 = start_times_bns[st2]
    bns_signal = bns_signal.taper()
    bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)

    data = noise.inject(bns_signal)
    data *= 1e-17

    data.write(directory + "merged_bns_noise_signal/bns_merged_noise_signal_"+str(i)+".txt")
np.savetxt(directory + "val_Final_BNS_Merged_Noise_Signal_Targets_"+str(no_of_params)+"_parameters.csv", bns_data_targets, delimiter = ",")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [11]:

path = directory + "merged_bns_noise_signal/"
files= os.listdir(path)
f = open(directory + 'val_Final_BNS_Merged_Noise_Signal_Reduced_No_ABS_'+str(no_of_params)+'_parameters.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
os.makedirs(directory+"noise/", exist_ok=True)

for i in tqdm(range(len(bbh_two_mass_samples))):

    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    # Generate 4 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(4 / delta_t)
    noise = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)

    noise *= 1e21
    noise *= 0.4
    noise = TimeSeries.from_pycbc(noise)
    
    noise *= 1e-17

    noise.write(directory + "noise/noise_"+str(i)+".txt")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
path = directory + "noise/"
files= os.listdir(path)
f = open(directory + 'val_Final_Merged_Noise_Reduced_No_ABS_' + str(no_of_params) + '_parameters.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]